In [10]:
%matplotlib inline
import pandas as pd
from scipy import stats
import seaborn as sns

import numpy as np
from collections import Counter

In [9]:
data=pd.read_csv("data/clean.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/clean.csv'

In [ ]:
df=data.copy()

In [ ]:
numeric = []
qualitative= []

for variable in df.columns.tolist():
    if df.dtypes[variable] in ['float64','int64']:
        numeric.append(variable)
    else :
        qualitative.append(variable)

##### Test de normalité

In [ ]:
def shapiro_test(x):
    try :
        res = stats.shapiro(x)
    except :
        return -1
    alpha = 0.05

    print("p = ",res.pvalue)

    if res.pvalue < alpha:  # null hypothesis: x comes from a normal distribution

        print("(shapiro)The null hypothesis can be rejected -> X ne possède pas une distribution normale")

    else:

        print("(shapiro) The null hypothesis cannot be rejected -> X possède éventuellement une distribution normale")

    return res

def omnibus_normaltest(x):
    #test D'Agostino-Pearson
    try :
        k2, p = stats.normaltest(x)
    except :
        return -1
    
    alpha = 0.05

    print("p = ",p)

    if p < alpha:  # null hypothesis: x comes from a normal distribution

        print("(normaltest) The null hypothesis can be rejected -> X ne possède pas une distribution normale")

    else:

        print("(normaltest) The null hypothesis cannot be rejected -> X possède éventuellement une distribution normale")

    return [k2,p]

## valeur missing

In [ ]:
def missing(df):
    df_null = pd.DataFrame([df.isnull().sum(),round(100*df.isnull().sum()/ len(df), 2), df.dtypes]).transpose().reset_index()
    df_null.columns = ["variable", "valeur_NA", "Pourcentage_NA", "type"]
    df_null = df_null[df_null.valeur_NA != 0].sort_values("valeur_NA",ascending = False).reset_index(drop = True)
    return df_null
missing(df)

In [ ]:
list(data.columns[data.isnull().sum() / len(data) > 0.75])

In [ ]:
d=data.copy()
df = d.drop('division_af',axis=1)

In [ ]:
d=list(data.columns[data.isnull().sum() / len(data) < 0.05])
for i in list(d):
    df = df.dropna(subset=[i], axis=0)
missing(df)

In [ ]:
df['fonction_af'] = df['fonction_af'].fillna(df['fonction_af'].mode()[0])
df['direction_af'] = df['direction_af'].fillna(df['direction_af'].mode()[0])
df['motif_affect'] = df['motif_affect'].fillna(df['motif_affect'].mode()[0])
df['service_af'] = df['service_af'].fillna(df['service_af'].mode()[0])
df['job_act'] = df['job_act'].fillna(df['job_act'].mode()[0])

d=list(df.columns[df.isnull().sum() / len(df) < 0.05])
for i in list(d):
    df = df.dropna(subset=[i], axis=0)
    
from sklearn.impute import KNNImputer
df[numeric]=KNNImputer(missing_values=np.nan, n_neighbors=3).fit_transform(df[numeric])
df[numeric] = np.round(df[numeric],decimals = 0) 

In [ ]:
missing(df)

In [ ]:
df['tot_afct'].plot(figsize=(14,10))

## Valeur aberante

In [ ]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

Outliers_to_drop = detect_outliers(df,0,numeric)
# df.drop(Outliers_to_drop,0,inplace=True)
len(Outliers_to_drop)

## Teste de normalite 

In [ ]:
# Shapiro-Wilk Test
from scipy.stats import shapiro
def Shapiro(d):
    stat, p = shapiro(d)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    
for i in numeric:
    print(i)
    Shapiro(df[i])

In [ ]:
from scipy.stats import normaltest
def agostino_test(d):
    stat, p = normaltest(d)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    
for i in numeric:
    agostino_test(df[i])

In [ ]:
p_values = {}
for col in numeric:
    print("\n",col)
    Shapiro(np.log(df[col].dropna()))

In [ ]:
len(df[qualitative[4]].unique())

In [ ]:
formation=['ECONOMIE CIRCULAIRE ET DEVELOPPEMENT DURABLE','CFEPENN','2EME REUNION DES EXPERTS DE','AGOA','FORMATION','SCANNER','RAPPROCHEMENT DES DONNEES COMPTABLES','RENFORCEMENT','SEMINAIRE','LUTTE','SURVEILLANCE','ATELIER','TECHNIQUE D\'ENQUETE','SCANNER','DACTYLOGRAPHIE','FOOD SAFETY INSPECTION TECHNOLOGY','COMMUNICATION INTERPERSONNELLE','ACCUEIL DES USAGERS','SURETE AEROPORTUAIRE','ATELIER SUR LES NEGOCIATIONS SUR LA ZLET TRIPARTITE','VALEUR TRANSACTIONNELLE','COMPTABILITE','BAE','MECANICIEN','CONTROLE A POSTERIORI','OPERATEUR IMAGE SCANNER','TABLEAU DE BORD','CORRESPONDANCE ADMINISTRATIVE','ATTACHE D\'ADMINISTRATION','REGLES D\'ORIGINE','SECRETARIAT','TEAM MANAGEMENT','DEVELOPPEMENT PERSONNEL','GESTION AXEE SUR LES RESULTATS','SECRETARIAT','MISE A NIVEAU','MECANIQUE AUTOMOBILE','MANAGEMENT ET LEADERSHIP','BUREAUTIQUE','LEADERSHIP','FORMATION']
MAITRISE=['DUES','4E ANNEE','BACC+4','BACC+5','SECOND CYCLE',"DIPLOME D'ETUDES PROFESSIONNELLES APPROFONDIES",'4EME ANNEE','MASTER','MAITRISE','ETHIQUE ET DEONTOLOGIE','DEA','MAGISTERE','INGENIORAT','INGENIEUR','2ND CYCLE','BACCALAUREAT+4']
LICENCE =['3EME ANNEE','BACC+3','LICENCE']
BREVET=['BREVET','CFEPCES','PREBAC','PRE BAC','BEPC']
PRIMAIRE=['CAE/EB','CEPE','CERTIFICAT D\'ETUDES PRIMAIRES ELEMENTAIRES']
DOUANE=['SYDONIA++','PROCEDURE DE DEDOUANEMENT','DOUAN','TRANSIT']
LANGUES=['B1-B2','ANGLAIS','FRANCAIS','DELF','ENGLISH','MTCP']
INFORMATIQUE=['ANALYSE D\'IMAGE','ATTESTATION','ELECTRONIQUE','ELECTROTECHNIQUE','PROGRAMMATION','DEVELOPPEUR','SOFTWARE','BASE DE DONNEE','ORACLE','EXCEL','WEB','LINUX','GRAPHISTE','RESEAU','INFORMATIQUE','PROGRAMMEUR']
PERMIS=['PERMIS','MAINTENANCE AUTOMOBILE','CHAUFFEUR','MECANIQUE AUTO','CONDUCTEUR']
BACC=['BACC','BACALAUREAT']
SUPERIEUR=['BT','BATIMENT','COMPTABLE','AUDIT INTERNE','DROIT','GESTION','DROIT / GEOGRAPHIE','MANAGEMENT','DIPLOME','INFA','IST-T','COMMUNICATION','SUPERIEUR','UNIVERSITAIRE','COACHING','CHEF','BANCAIRES','INTERNATIONALES','ETUDES APPROFONDIES','SERVICE NATIONAL HORS FORCES ARMEES','RADIOPROTECTION','ETUDES JUDICIAIRES','CONTROLE A LA CIRCULATION','MARCHE PUBLIC','ENVIRONNEMENT','OZONE','SURETE AEROPORTUAIRE ET FRAUDE DOCUMENTAIRE','SUBSTANCES APPAUVRISSANT LA COUCHE D\'OZONE','COMMERCE INTERNATIONAL','CITES','CONTROLE DES SUBSTANCES APPAUVRISSANT LA COUCHE D\'OZONE','DIPLOME D\'ATTACHE D\'ADMINISTRATION']
DTS=['2E ANNEE','DEUG','2EME ANNEE','BACC+2','DTS']
Autres=['FORMATION','MAITRISE','LICENCE','BREVET','PRIMAIRE','DOUANE','LANGUES','INFORMATIQUE','PERMIS','BACC','SUPERIEUR','DTS']
dat = list(df[qualitative[4]])
for index, value in enumerate(dat):
    for i in DTS:
        if i in str(value):
            dat[index]='DTS'
    for i in BACC:
        if i in str(value):
            dat[index]='BACC'
    for i in SUPERIEUR:
        if i in str(value):
            dat[index]='SUPERIEUR'
    for i in PERMIS:
        if i in str(value):
            dat[index]='PERMIS'
    for i in INFORMATIQUE:
        if i in str(value):
            dat[index]='INFORMATIQUE'
    for i in LANGUES:
        if i in str(value):
            dat[index]='LANGUES'
    for i in DOUANE:
        if i in str(value):
            dat[index]='DOUANE'
    for i in PRIMAIRE:
        if i in str(value):
            dat[index]='PRIMAIRE'
    for i in BREVET:
        if i in str(value):
            dat[index]='BREVET'
    for i in LICENCE:
        if i in str(value):
            dat[index]='LICENCE'
    for i in formation:
        if str(i) in str(value):
            dat[index]='FORMATION'
    for i in MAITRISE:
        if i in str(value):
            dat[index]='MAITRISE'
    if 'CERTIFICAT' in str(value):
        dat[index]='CERTIFICAT'
    if 'DOCTORAT' in str(value):
        dat[index]='DOCTORAT'
        
df[qualitative[4]] =  dat
d = list(df[qualitative[4]])
for index, value in enumerate(d):
    if  str(value) not in Autres:
        print(value)
        d[index]='Autres'
df[qualitative[4]] =  d


In [ ]:
def processPerLevels(data,list_feature):
    catHead = ['Feature', 'Level', 'Count', 'Frequence %']
    count = []
    freq = []
    feature_all = []
    list_level_all = []
    for feature in list_feature:
        
        list_level = data[feature].unique().tolist()

        len_level = len(list_level)

        for level in list_level:
            count_level = len(data.loc[data[feature] == level])
            count.append(count_level)
            freq_level = count_level * 100/len(data)
            freq.append(round(freq_level, 2))
            feature_all.append(feature)
            list_level_all.append(level)
    
    resultDf = pd.DataFrame(columns=catHead)
    resultDf['Level'] = list_level_all
    resultDf['Feature'] = feature_all
    resultDf['Count'] = count
    resultDf['Frequence %'] = freq
    resultDf.set_index(['Feature', 'Level'], inplace=True)
    return resultDf
processPerLevels(df,[qualitative[4]])

In [ ]:
print(len(df[qualitative[15]].unique()))
processPerLevels(df,[qualitative[15]])

## log

## Stendardisation

In [ ]:
df.to_csv(r'data/preproce.csv',index=False)

